# Split dataset en fichiers de 50 mo afin d'être compatible avec github

In [1]:
# import os
# import pandas as pd

# def split_dataset_50mo(dataset, output_dir="dataset_50mo"):
#     """
#     Divise un dataset en plusieurs fichiers plus petits.

#     Paramètres :
#     - dataset (str) : Le chemin vers le fichier du dataset.
#     - output_dir (str) : Le répertoire pour stocker les fichiers divisés.
#     - chunk_size (int) : Le nombre de lignes par fichier divisé.
#     """
#     chunk_size=50000
#     try:
#         # Crée le répertoire de sortie s'il n'existe pas
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)

#         # Détermine le format du fichier
#         _, file_extension = os.path.splitext(dataset)
#         file_format = file_extension[1:]

#         # Lit et divise le dataset
#         chunk_number = 0
#         for chunk in pd.read_csv(dataset, chunksize=chunk_size):
#             chunk_file = os.path.join(output_dir, f"chunk_{chunk_number}.{file_format}")
#             chunk.to_csv(chunk_file, index=False)
#             chunk_number += 1
#     except Exception as e:
#         print(f"Une erreur est survenue lors de la division du dataset : {e}")



In [14]:
# #  test
# dataset = 'Station44_test.csv'
# dossier_cible="Datasets/dataset_splited_50mo"
# split_dataset_50mo(dataset, dossier_cible)


In [55]:
import os
import pandas as pd

def split_dataset(dataset, chunk_size=50000):
    """
    Divise un dataset en plusieurs fichiers plus petits.

    Paramètres :
    - dataset (str) : Le chemin vers le fichier du dataset.
    - chunk_size (int) : Le nombre de lignes ou d'enregistrements par fichier divisé.
    """
    try:
        # Récupère le nom de base du fichier sans l'extension
        base_name = os.path.basename(dataset)
        base_name_without_ext = os.path.splitext(base_name)[0]

        # Crée le répertoire de sortie nommé d'après le fichier original suivi de "_splitted"
        output_dir = f"{base_name_without_ext}_splitted"
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        # Détermine le format du fichier et sélectionne la méthode de lecture appropriée
        _, file_extension = os.path.splitext(dataset)
        if file_extension.lower() == '.csv':
            read_function = pd.read_csv
            save_function = pd.DataFrame.to_csv
            read_args = {}
            save_args = {'index': False}
        elif file_extension.lower() == '.json':
            read_function = pd.read_json
            save_function = pd.DataFrame.to_json
            read_args = {'lines': True}
            save_args = {'orient': 'records', 'lines': True}
        elif file_extension.lower() == '.txt' or file_extension.lower() == '.tsv':
            read_function = pd.read_csv
            save_function = pd.DataFrame.to_csv
            read_args = {'sep': '\t'}
            save_args = {'sep': '\t', 'index': False}
        else:
            raise ValueError(f"Format de fichier non supporté : {file_extension}")

        # Lit et divise le dataset
        chunk_number = 0
        for chunk in read_function(dataset, chunksize=chunk_size, **read_args):
            chunk_file = os.path.join(output_dir, f"{base_name_without_ext}_{chunk_number}{file_extension}")
            save_function(chunk, chunk_file, **save_args)
            chunk_number += 1
        print(f"Dataset divisé en {chunk_number} fichiers dans le répertoire '{output_dir}'.")
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"Une erreur est survenue lors de la division du dataset : {e}")

# Exemple d'utilisation
# split_dataset('chemin/vers/le/fichier/Station44_test.csv', chunk_size=100)
# split_dataset('chemin/vers/le/fichier/data.json', chunk_size=100)


In [58]:
dataset = 'Station44_test.csv'
# dataset = 'clean_urls.json'
# dataset = 'primer-dataset.json'
# dossier_cible="Datasets/dataset_splited_50mo"
# split_dataset_50mo(dataset)
split_dataset_50mo(dataset, chunk_size=500000)


Dataset divisé en 3 fichiers dans le répertoire 'Station44_test_splitted'.


In [72]:
import os
import pandas as pd

def unsplit_dataset(output_dir):
    """
    Combine plusieurs fichiers plus petits en un seul dataset et le nomme d'après le répertoire d'origine.

    Paramètres :
    - output_dir (str) : Le répertoire contenant les fichiers divisés.
    """

    try:
        # Vérifie si le répertoire de sortie existe
        if not os.path.exists(output_dir):
            raise FileNotFoundError(f"Le répertoire spécifié n'existe pas : {output_dir}")

        # Obtient une liste de tous les fichiers divisés
        chunk_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if not f.startswith('.')]

        # Trie les fichiers divisés
        chunk_files.sort()

        # Vérifie si des fichiers à combiner ont été trouvés
        if not chunk_files:
            raise FileNotFoundError(f"Aucun fichier à combiner n'a été trouvé dans : {output_dir}")

        # Déduit l'extension de fichier pour déterminer la méthode de lecture et d'écriture
        _, file_extension = os.path.splitext(chunk_files[0])

        # Crée le nom pour le fichier de sortie
        base_output_dir = os.path.basename(output_dir)
        base_name_unsplitted = base_output_dir.replace("_splitted", "_unsplitted")
        output_file = f"{base_name_unsplitted}{file_extension}"

        # Détermine la méthode d'écriture en fonction de l'extension du fichier
        if file_extension.lower() == '.json':
            df_list = [pd.read_json(f, lines=True) for f in chunk_files]
            df = pd.concat(df_list)
            df.to_json(output_file, orient='records', lines=True)
        elif file_extension.lower() == '.csv':
            df_list = [pd.read_csv(f) for f in chunk_files]
            df = pd.concat(df_list)
            df.to_csv(output_file, index=False)
        elif file_extension.lower() == '.txt' or file_extension.lower() == '.tsv':
            sep = '\t' if file_extension.lower() == '.tsv' else ' '
            df_list = [pd.read_csv(f, sep=sep) for f in chunk_files]
            df = pd.concat(df_list)
            df.to_csv(output_file, sep=sep, index=False)
        else:
            raise ValueError(f"Extension de fichier non prise en charge : {file_extension}")

        print(f"Dataset combiné avec succès et enregistré sous : {output_file}")
    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e)
    except Exception as e:
        print(f"Une erreur est survenue lors de la combinaison des fichiers : {e}")

# Exemple d'utilisation
# dossier_cible = "Station44_test_splitted"
# unsplit_dataset(dossier_cible)


In [73]:
#  test unsplit
# dossier_cible="Station44_test_splitted"
dossier_cible="primer-dataset_splitted"
unsplit_dataset(output_dir=dossier_cible)


Dataset combiné avec succès et enregistré sous : primer-dataset_unsplitted.json


# Test des differences entre les datasets apres reconstitution

In [74]:
df1=pd.read_csv("Station44_test.csv")
df2=pd.read_csv("Station44_test_unsplitted.csv")

print(f"\ndf1 :\n{df1.shape} \n")
print(f"\n df2 :\n{df2.shape} \n")

sont_identiques = df1.equals(df2)
print("Les deux datasets sont identiques :", sont_identiques)

# Fusionner les deux DataFrames et indiquer l'origine de chaque ligne
df = pd.merge(df1, df2, how='outer', indicator=True)

# Trouver les lignes qui sont différentes
differences = df[df['_merge'] != 'both']

print("Les différences entre les deux datasets sont :")
print(differences)

differences_df1 = differences[differences['_merge'] == 'left_only']

print(f"\ndifferences_df1 :\n{differences_df1} \n")

differences_df2 = differences[differences['_merge'] == 'right_only']
print(f"\n differences_df2:\n{differences_df2} \n")

# print(f"df1  {df1.describe()}")
print(f"df1 colonnes  {df1.columns.tolist()}")
# print(f"df2  {df2.describe()}")
print(f"df2 colonnes  {df2.columns.tolist()}")



df1 :
(1297315, 1) 


 df2 :
(1297315, 1) 

Les deux datasets sont identiques : True
Les différences entre les deux datasets sont :
Empty DataFrame
Columns: [Station Id;Date Timestamp [UTC];Date Txt [UTC]; Average Wind Speed [nds];Min Wind Speed [nds];Max Wind Speed [nds];Wind Direction [degree];Wind Direction [txt], _merge]
Index: []

differences_df1 :
Empty DataFrame
Columns: [Station Id;Date Timestamp [UTC];Date Txt [UTC]; Average Wind Speed [nds];Min Wind Speed [nds];Max Wind Speed [nds];Wind Direction [degree];Wind Direction [txt], _merge]
Index: [] 


 differences_df2:
Empty DataFrame
Columns: [Station Id;Date Timestamp [UTC];Date Txt [UTC]; Average Wind Speed [nds];Min Wind Speed [nds];Max Wind Speed [nds];Wind Direction [degree];Wind Direction [txt], _merge]
Index: [] 

df1 colonnes  ['Station Id;Date Timestamp [UTC];Date Txt [UTC]; Average Wind Speed [nds];Min Wind Speed [nds];Max Wind Speed [nds];Wind Direction [degree];Wind Direction [txt]']
df2 colonnes  ['Station Id;Date 